In [34]:
import numpy as np
import numpy_indexed as npi
import sys
import itertools
from itertools import permutations 
from itertools import product
from collections import Counter
import scipy.io as sio
import pickle
import copy

In [35]:
import sbd
import cluster_calcs
import dynamics
import direct

First, we provide a list of edges:

In [45]:
edges_2 = [[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[1,7],
           [1,3],[2,4],[3,5],[4,6],[5,7],[1,6],[2,7],
           [1,8],[2,9],[3,10],[4,11],[5,12],[6,13],[7,14],
           [8,9],[9,10],[10,11],[11,12],[12,13],[13,14],[8,14],
           [8,10],[9,11],[10,12],[11,13],[12,14],[8,13],[9,14]]
edges_3 = [[1,2,3],[2,3,4],[3,4,5],[4,5,6],[5,6,7],[1,6,7],[1,2,7],
           [8,9,10],[9,10,11],[10,11,12],[11,12,13],[12,13,14],[8,13,14],[8,9,14]]
clusters = [[1,2,3,4,5,6,7],[8,9,10,11,12,13,14]]

edges = {}
edges['2'] = edges_2
edges['3'] = edges_3

clust = cluster_calcs.ClusterSync(edges, clusters) 

Simultaneous block diagonalization:

In [46]:
err = 1e-8
clust.SBD(err)
print('Dimensions of transverse blocks are:' + str(clust.block_size_SBD_transverse))

Dimensions of transverse blocks are:[4 4 4]


In [41]:
#method_list = [func for func in dir(clust) if callable(getattr(clust, func)) and not func.startswith("__")]
#dir(clust)

And save the results:

In [47]:
pickle.dump(clust, open("cluster_sync.p", "wb"))

Now, we define the dynamical terms and their partial derivatives with respect to contributing nodes (assuming Laplacian-like coupling). The maximum edge order is arbitrary.

This is valid for $\dot{x_i}=...+G^{(m)}(\sum\limits_mx_{j_{m}}-m x_i)$, where $\dfrac{\partial G^{(m)}(\sum x_{j_{m}}-m x_i)}{\partial j_k}$ are equal.

In [29]:
def self_term(x):
    """Find the value of the self-term"""
    return (np.sin(x + np.pi/4))**2
    
def dyadic_term(x):
    """Find the value of the dyadic coupling term"""
    return (np.sin(x[1] + np.pi/4))**2 - (np.sin(x[0] + np.pi/4))**2
    
def triadic_term(x):
    """Find the value of the triadic term"""
    return (- np.sin(x[1] + x[2] - 2*x[0]))
    
def self_term_J(x):
    """Partial derivative of the self-term"""
    return np.sin(2*x + np.pi/2)
        
def dyadic_term_J(x):
    """Partial derivative of the dyadic term"""
    return np.sin(2*x[1] + np.pi/2)
        
def triadic_term_J(x):
    """Partial derivative of the triadic term"""
    return (- np.cos(x[1] + x[2] - 2*x[0]))

dynamics_terms = {}
dynamics_terms['1'] = self_term
dynamics_terms['2'] = dyadic_term
dynamics_terms['3'] = triadic_term
 
jacobian_terms = {}
jacobian_terms['1'] = self_term_J
jacobian_terms['2'] = dyadic_term_J
jacobian_terms['3'] = triadic_term_J

Define the parameter ranges:

In [30]:
sigma_2_max = .6
n_2 = 7 # number of sigma_2 parameters
sigma_3_max = .4
n_3 = 7 # number of sigma_3 parameters
beta = 2.3

coupling_strengths = {}
coupling_strengths['2'] = np.arange(n_2) * sigma_2_max/n_2
coupling_strengths['3'] = np.arange(n_3) * sigma_3_max/n_3

And the number of steps:

In [31]:
T = 2000

Evolve the dynamics on quotient networks together with perturbations:

In [32]:
dyn_shape = ()
for order in clust.orders:
    dyn_shape+= (len(coupling_strengths[order]),)
norm_shape = copy.copy(dyn_shape)
dyn_shape+= (T,)
norm_shape+= (T-1,)
perturb_shape = copy.copy(dyn_shape)
dyn_shape+= (clust.nclusters['1'],)
perturb_shape+= (clust.size - clust.nclusters['1'],)
dynamics_quotient = np.zeros(dyn_shape)
dynamics_perturbations = np.zeros(perturb_shape)
norms = np.zeros(norm_shape)
    
iter_inds = []
for order in clust.orders:
    iter_inds+= [(np.arange(len(coupling_strengths[order]))).tolist()]
    
for ind in product(*iter_inds):
    sigma = {}
    for i, order in enumerate(clust.orders):
        sigma[order] = coupling_strengths[order][ind[i]]
    dyn = dynamics.Dynamics(clust, dynamics_terms, jacobian_terms, beta, sigma)
    dyn_quotient, dyn_perturbations, norm = dyn.extended_dynamics_evolution(T)
    dynamics_quotient[ind] = dyn_quotient # evolution of the state of each cluster
    dynamics_perturbations[ind] = dyn_perturbations # perturbation dynamics
    norms[ind] = norm

And save the results:

In [33]:
pickle.dump(norms, open("norms.p", "wb"))
pickle.dump(dynamics_quotient, open("quotient.p", "wb"))
pickle.dump(dynamics_perturbations, open("perturbations.p", "wb"))
quotient_last = quotient[:,:,-1,:]
pickle.dump(quotient_last, open("quotient_last.p", "wb"))